In [163]:
import numpy as np
import pandas as pd
import math

In [164]:
articles_df = pd.read_csv("./data/shared_articles.csv")
interactions_df = pd.read_csv("./data/users_interactions.csv")

In [165]:
interactions_df.personId = interactions_df.personId.astype(str)
interactions_df.contentId = interactions_df.contentId.astype(str)
articles_df.contentId = articles_df.contentId.astype(str)

In [166]:
articles_df.query("eventType == 'CONTENT SHARED'").count()

timestamp          3047
eventType          3047
contentId          3047
authorPersonId     3047
authorSessionId    3047
authorUserAgent     669
authorRegion        669
authorCountry       669
contentType        3047
url                3047
title              3047
text               3047
lang               3047
dtype: int64

In [167]:
# Задание 6.2

# Создайте признак, который будет отражать числовой вес для взаимодействия со статьёй (в соответствии с приведёнными выше весами).
# Вычислите среднее значение для полученного признака. Округлите его до двух знаков после точки-разделителя.

event_type = {
    'VIEW': 1.0,
    'LIKE': 2.0,
    'BOOKMARK': 2.5,
    'FOLLOW': 3.0,
    'COMMENT CREATED': 4.0,
}

interactions_df['event_code'] = interactions_df['eventType'].map(event_type)
interactions_df['event_code'].mean().round(2)


1.24

In [168]:
interactions_df.columns

Index(['timestamp', 'eventType', 'contentId', 'personId', 'sessionId',
       'userAgent', 'userRegion', 'userCountry', 'event_code'],
      dtype='object')

In [169]:
# Задание 6.3

# Чтобы получить хоть какую-то информацию, на которую можно будет опираться, оставьте только тех пользователей, которые
# взаимодействовали хотя бы с пятью статьями. Сколько всего таких пользователей?
interactions_counter = interactions_df.groupby(
    by=['personId', 'contentId']).first().reset_index().groupby(by=['personId']).size()

users_with_enough_interactions_df = \
    interactions_counter[interactions_counter >= 5].reset_index()

users_with_enough_interactions_df.rename(columns={0: "counter"}, inplace=True)

users_with_enough_interactions_df.shape[0]


1140

In [170]:
# Задание 6.4

# Теперь оставим только те взаимодействия, которые касаются только отфильтрованных пользователей (то есть тех, которые взаимодействовали
# как минимум с пятью статьями). Сколько всего таких взаимодействий?

mask = users_with_enough_interactions_df['personId']
selected_interactions_df = interactions_df[interactions_df['personId'].isin(mask)]
selected_interactions_df.shape[0]

69868

In [172]:
# Задание 6.5

# Примените упомянутое выше преобразование для логарифмирования к сумме весов для взаимодействия пользователя с каждой конкретной
# статьёй. Также сохраните для каждой пары «пользователь - статья» значение времени последнего взаимодействия.
# Найдите среднее по признаку с получившимися временными отсечками. Округлите результат до двух знаков после точки-разделителя.

def smooth_user_preference(x):
    return math.log(1+x, 2)


sum_interactions_log = selected_interactions_df.groupby(by=['personId', 'contentId'])[
    'event_code'].sum().apply(smooth_user_preference).reset_index()

sum_interactions_log = pd.DataFrame(sum_interactions_log).rename(
    columns={"event_code": "sum_attention"})

max_timestamp = selected_interactions_df.groupby(by=['personId', 'contentId'])[
    'timestamp'].max().reset_index()

sum_interactions_log = pd.merge(sum_interactions_log, max_timestamp, on=[
                                'personId', 'contentId'])
sum_interactions_log = sum_interactions_log.reindex(
    columns=['personId', 'contentId', 'timestamp', 'sum_attention'])

sum_interactions_log['timestamp'].mean().round(2)


1470605340.04

In [182]:
# Задание 6.6

# Разделите данные на обучающую и тестовую выборки. выбрав в качестве временной отсечки значение 1475519545. Значение отсечки включите
# в тестовую выборку. Сколько объектов попало в обучающую выборку?
sum_interactions_log = sum_interactions_log.sort_values(by='timestamp')

time_gate = 1475519545

mask = sum_interactions_log['timestamp'] >= time_gate

train_df = sum_interactions_log[sum_interactions_log['timestamp'].isin(~mask)]
test_df = sum_interactions_log[sum_interactions_log['timestamp'].isin(mask)]

In [183]:
train_df

,personId,contentId,timestamp,sum_attention


In [184]:
test_df

,personId,contentId,timestamp,sum_attention


In [186]:
sum_interactions_log['timestamp'].isin(mask)

34061    False
13341    False
10929    False
11381    False
19304    False
         ...  
8654     False
36116    False
1755     False
28206    False
8590     False
Name: timestamp, Length: 39106, dtype: bool